In [6]:
# Data Collection & Combination - CMS Hospital Reports (2011–2022)

import os
import pandas as pd

# Define paths
raw_dir = "cms_raw"
combined_dir = "combined"
output_file = os.path.join(combined_dir, "cms_hospital_costs_2011_2022.csv")

# Step 1: Detect all CSV files

files = [f for f in os.listdir(raw_dir) if f.endswith(".csv")]
files.sort()

print(f"\n 1. Detecting CSV files in '{raw_dir}' ...")
print(f"{len(files)} CSV files detected:\n")
for f in files:
    print(" -", f)

# Step 2: Check column consistency before merging

column_map = {}
for file in files:
    df_temp = pd.read_csv(os.path.join(raw_dir, file), nrows=5)
    column_map[file] = df_temp.columns.tolist()

unique_col_counts = {file: len(cols) for file, cols in column_map.items()}

print("\n Column count per file:")
for name, count in unique_col_counts.items():
    print(f" {name}: {count} columns")

if len(set(unique_col_counts.values())) == 1:
    print("\n All files have the same number of columns.")
else:
    print("\n Warning: Some files have different column counts!")

    # Find differing columns
    all_columns = set().union(*column_map.values())
    for file, cols in column_map.items():
        missing = all_columns - set(cols)
        extra = set(cols) - all_columns
        if missing:
            print(f"    {file} is missing {len(missing)} columns: {list(missing)[:5]} ...")
        if extra:
            print(f"    {file} has extra {len(extra)} columns: {list(extra)[:5]} ...")


# Step 3: Read and combine all CSVs
print("\nStep 3: Combining all datasets...")

combined_data = pd.DataFrame()

for file in files:
    year = file.split("_")[1] if "_" in file else "Unknown"
    file_path = os.path.join(raw_dir, file)
    df = pd.read_csv(file_path)
    df['year'] = year  # add year column
    combined_data = pd.concat([combined_data, df], ignore_index=True)
    print(f" Added: {file} ({df.shape[0]:,} rows, {df.shape[1]} columns)")

# Step 4: Save final combined dataset
os.makedirs(combined_dir, exist_ok=True)
combined_data.to_csv(output_file, index=False)

print("\n Data combination complete!")
print(f" Saved to: {output_file}")
print(f" Final dataset shape: {combined_data.shape[0]:,} rows × {combined_data.shape[1]} columns")

# Step 5: Quick sanity check

print("\n Preview of final combined dataset:")
display(combined_data.head())

print("\n Summary by year:")
print(combined_data['year'].value_counts().sort_index())



 1. Detecting CSV files in 'cms_raw' ...
12 CSV files detected:

 - CostReport_2011_Final.csv
 - CostReport_2012_Final.csv
 - CostReport_2013_Final.csv
 - CostReport_2014_Final.csv
 - CostReport_2015_Final.csv
 - CostReport_2016_Final.csv
 - CostReport_2017_Final.csv
 - CostReport_2018_Final.csv
 - CostReport_2019_Final.csv
 - CostReport_2020_Final.csv
 - CostReport_2021_Final.csv
 - CostReport_2022_Final.csv

 Column count per file:
 CostReport_2011_Final.csv: 117 columns
 CostReport_2012_Final.csv: 117 columns
 CostReport_2013_Final.csv: 117 columns
 CostReport_2014_Final.csv: 117 columns
 CostReport_2015_Final.csv: 117 columns
 CostReport_2016_Final.csv: 117 columns
 CostReport_2017_Final.csv: 117 columns
 CostReport_2018_Final.csv: 117 columns
 CostReport_2019_Final.csv: 117 columns
 CostReport_2020_Final.csv: 117 columns
 CostReport_2021_Final.csv: 117 columns
 CostReport_2022_Final.csv: 117 columns

 All files have the same number of columns.

Step 3: Combining all datasets...
 

,rpt_rec_num,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone CHIP,Stand-Alone CHIP Charges,year
0,285,10005,MARSHALL MEDICAL CENTER - SOUTH,2505 U.S. HIGHWAY 431,BOAZ,AL,35957-,MARSHALL,13820.0,R,...,5331504.0,-2691671.0,NaN,-2691671.0,0.282971,8362465.0,30633130.0,NaN,NaN,2011
1,1022,271326,BEARTOOTH BILLINGS CLINIC,2525 NORTH BROADWAY,RED LODGE,MT,59806,CARBON,99927.0,R,...,725351.0,-837408.0,NaN,-837408.0,1.061421,186460.0,262576.0,NaN,NaN,2011
2,1496,10052,LAKE MARTIN COMMUNITY HOSPITAL,1231 SOUTH STREET,DADEVILLE,AL,36853,TALLAPOOSA,99919.0,U,...,271686.0,132630.0,NaN,132630.0,0.641784,287318.0,1141082.0,NaN,NaN,2011
3,1501,13025,HEALTHSOUTH LAKESHORE HOSPITAL,3800 RIDGEWAY DRIVE,BIRMINGHAM,AL,35209,JEFFERSON,13820.0,U,...,421791.0,11784376.0,21729.0,11762647.0,0.452421,NaN,NaN,NaN,NaN,2011
4,1504,103037,HEALTHSOUTH REHABILITATION HOSPITAL,901 NORTH CLEARWATER-LARGO ROAD,LARGO,FL,33770,PINELLAS,45300.0,U,...,56946.0,4565066.0,NaN,4565066.0,0.663245,NaN,NaN,NaN,NaN,2011



 Summary by year:
year
2011    6150
2012    6227
2013    6248
2014    6250
2015    6257
2016    6211
2017    6174
2018    6160
2019    6121
2020    6059
2021    6053
2022    6064
Name: count, dtype: int64
